## Impressions
### view_attributed_units_sold is atleast 1

## Data

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load the Excel file to explore its content and structure
campaign_data = pd.read_csv('Amazon Ads All.csv')

# Display the first few rows of the dataframe and the column names to understand the dataset
campaign_data.head(), campaign_data.columns

(   ADV_ID    AD_ID         SKU     placement_slot  impressions  clicks  \
 0   84373  6370624  9985955750  offsite_mobileweb         1089       0   
 1   84373  1295011  7875390725    offsite_desktop         2043       0   
 2   84373  6370624  9985955750  offsite_mobileweb         1452       0   
 3   84373  1295011  7875390725    offsite_desktop         1312       0   
 4   84373  1295011  7875390725  offsite_mobileapp            8       0   
 
    auction_cost  adjusted_cost hit_day_utc  \
 0      10.89000        8.79000    02/05/24   
 1      26.61464       19.44791    01/05/24   
 2      14.52000       11.68400    01/05/24   
 3      16.59940       11.94286    30/04/24   
 4       0.11608        0.07161    01/05/24   
 
                                  targeting_secondary    vertical  \
 0       category=Automotive Replacement Control Arms  Automotive   
 1         category=Automotive Replacement Brake Kits  Automotive   
 2       category=Automotive Replacement Control Arms  Au

## Cleaning

In [2]:
## CLEANING
# Attempt to parse dates in the '%d/%m/%y' format
campaign_data['hit_day_utc_parsed'] = pd.to_datetime(campaign_data['hit_day_utc'], format='%d/%m/%y', errors='coerce')

# Parse remaining unparsed dates in the '%Y-%m-%d %H:%M:%S' format
mask = campaign_data['hit_day_utc_parsed'].isna()
campaign_data.loc[mask, 'hit_day_utc_parsed'] = pd.to_datetime(campaign_data.loc[mask, 'hit_day_utc'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Check for any remaining non-parsed dates
remaining_non_parsed = campaign_data[campaign_data['hit_day_utc_parsed'].isna()]
if not remaining_non_parsed.empty:
    print("Remaining non-parsed dates:")
    print(remaining_non_parsed)

# Print unique months in the parsed 'hit_day_utc_parsed' column
print("Unique months in 'hit_day_utc_parsed':")
print(campaign_data['hit_day_utc_parsed'].dt.month.unique())

# Extract month and day of the week from parsed dates

campaign_data['month_name'] = campaign_data['hit_day_utc_parsed'].dt.strftime('%B')  # Get month names
campaign_data['day_name'] = campaign_data['hit_day_utc_parsed'].dt.strftime('%A')    # Get day names

# Fill missing values in 'vertical' and 'sub_vertical' with 'Unknown'
campaign_data['vertical'].fillna('Unknown', inplace=True)
campaign_data['sub_vertical'].fillna('Unknown', inplace=True)

# Identify the last 6 columns
last_6_columns = campaign_data.columns[-6:]

# Fill missing values in the last 6 columns with 0
campaign_data[last_6_columns] = campaign_data[last_6_columns].fillna(0)

# Display the cleaned dataset
print("Cleaned Campaign Data Sample:")
print(campaign_data.head())

# Inspect the 'hit_day_utc' column to understand its format
print("Sample of 'hit_day_utc' column:")
print(campaign_data['hit_day_utc'].head(10))


# Filter the data where view_attributed_orders is at least 1
filtered_data = campaign_data[campaign_data['view_attributed_units_sold'] >= 1]

Unique months in 'hit_day_utc_parsed':
[5 4 3 2]


/var/folders/80/ghdt2w012_x2yg018vx45m300000gn/T/ipykernel_61538/3580200166.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  campaign_data['vertical'].fillna('Unknown', inplace=True)
/var/folders/80/ghdt2w012_x2yg018vx45m300000gn/T/ipykernel_61538/3580200166.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting 

Cleaned Campaign Data Sample:
   ADV_ID    AD_ID         SKU     placement_slot  impressions  clicks  \
0   84373  6370624  9985955750  offsite_mobileweb         1089       0   
1   84373  1295011  7875390725    offsite_desktop         2043       0   
2   84373  6370624  9985955750  offsite_mobileweb         1452       0   
3   84373  1295011  7875390725    offsite_desktop         1312       0   
4   84373  1295011  7875390725  offsite_mobileapp            8       0   

   auction_cost  adjusted_cost hit_day_utc  \
0      10.89000        8.79000    02/05/24   
1      26.61464       19.44791    01/05/24   
2      14.52000       11.68400    01/05/24   
3      16.59940       11.94286    30/04/24   
4       0.11608        0.07161    01/05/24   

                                 targeting_secondary  ...        sub_vertical  \
0       category=Automotive Replacement Control Arms  ...  Automotive - Other   
1         category=Automotive Replacement Brake Kits  ...  Automotive - Other   
2    

## Model

### Train with all columns

In [7]:
# Prepare the dataset

# Display columns to ensure the correct ones are present
print("Columns in Campaign Data:")
print(filtered_data.columns)

# Check if 'placement_slot', 'targeting_secondary', 'vertical', and 'sub_vertical' exist in the columns before encoding
categorical_columns = ['placement_slot', 'targeting_secondary', 'vertical', 'sub_vertical', 'day_name', 'month_name']
existing_categorical_columns = [col for col in categorical_columns if col in filtered_data.columns]

if existing_categorical_columns:
    filtered_data = pd.get_dummies(filtered_data, columns=existing_categorical_columns, drop_first=True)

# Select features and target variable
features = ['clicks'] + \
           [col for col in filtered_data.columns if col.startswith('placement_slot_')] + \
           [col for col in filtered_data.columns if col.startswith('targeting_secondary_')] + \
           [col for col in filtered_data.columns if col.startswith('vertical_')] + \
           [col for col in filtered_data.columns if col.startswith('sub_vertical_')] + \
           [col for col in filtered_data.columns if col.startswith('month_name_')] + \
           [col for col in filtered_data.columns if col.startswith('day_name_')]

target = 'impressions'

X = filtered_data[features]
y = filtered_data[target]

# Handle missing values using SimpleImputer
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

if len(numeric_features) > 0:
    numeric_imputer = SimpleImputer(strategy='mean')
    X[numeric_features] = numeric_imputer.fit_transform(X[numeric_features])

if len(categorical_features) > 0:
    categorical_imputer = SimpleImputer(strategy='most_frequent')
    X[categorical_features] = categorical_imputer.fit_transform(X[categorical_features])

# Check if any missing values remain
print("Missing values in X after imputation:")
print(X.isna().sum().sum())

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Gradient Boosting Regressor
gbr = GradientBoostingRegressor(random_state=42)
gbr.fit(X_train, y_train)

# Predict on the test set
y_pred = gbr.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Display the evaluation metrics
evaluation_metrics = {
    'Mean Absolute Error': mae,
    'Mean Squared Error': mse,
    'R-squared': r2
}

print("Evaluation Metrics:")
print(pd.DataFrame([evaluation_metrics]))

# Display feature importances
feature_importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': gbr.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importance for Gradient Boosting Regressor:")
pd.set_option('display.max_rows', None)  # Ensure all rows are displayed
print(feature_importance_df)

Columns in Campaign Data:
Index(['ADV_ID', 'AD_ID', 'SKU', 'impressions', 'clicks', 'auction_cost',
       'adjusted_cost', 'hit_day_utc', 'ntb_click_attributed_units_sold',
       'ntb_view_attributed_units_sold',
       ...
       'sub_vertical_Wireless Products', 'day_name_Monday',
       'day_name_Saturday', 'day_name_Sunday', 'day_name_Thursday',
       'day_name_Tuesday', 'day_name_Wednesday', 'month_name_February',
       'month_name_March', 'month_name_May'],
      dtype='object', length=1038)


/var/folders/80/ghdt2w012_x2yg018vx45m300000gn/T/ipykernel_42389/4206416544.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numeric_features] = numeric_imputer.fit_transform(X[numeric_features])


Missing values in X after imputation:
0
Evaluation Metrics:
   Mean Absolute Error  Mean Squared Error  R-squared
0          1650.389724        7.136185e+06   0.659659

Feature Importance for Gradient Boosting Regressor:
                                                Feature    Importance
1                      placement_slot_offsite_mobileapp  3.400056e-01
922   targeting_secondary_category=Women's Running S...  2.426638e-01
0                                                clicks  1.142206e-01
346      targeting_secondary_category=Foundation Makeup  3.984460e-02
966     targeting_secondary_content-category=Soap Opera  3.481670e-02
2                      placement_slot_offsite_mobileweb  2.324876e-02
985                                 sub_vertical_Beauty  1.756840e-02
611     targeting_secondary_category=Oral Care Products  1.427540e-02
1016                                   month_name_March  1.397841e-02
722   targeting_secondary_category=Sheet & Pillowcas...  1.352223e-02
424      

In [8]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
print(feature_importance_df)

                                                                             Feature  \
1                                                   placement_slot_offsite_mobileapp   
922                               targeting_secondary_category=Women's Running Shoes   
0                                                                             clicks   
346                                   targeting_secondary_category=Foundation Makeup   
966                                  targeting_secondary_content-category=Soap Opera   
2                                                   placement_slot_offsite_mobileweb   
985                                                              sub_vertical_Beauty   
611                                  targeting_secondary_category=Oral Care Products   
1016                                                                month_name_March   
722                             targeting_secondary_category=Sheet & Pillowcase Sets   
424                             

### Train without vertical sub verticals

In [3]:
# Prepare the dataset

# Display columns to ensure the correct ones are present
print("Columns in Campaign Data:")
print(filtered_data.columns)

# Check if 'placement_slot', 'targeting_secondary', 'vertical', and 'sub_vertical' exist in the columns before encoding
categorical_columns = ['placement_slot', 'targeting_secondary', 'day_name', 'month_name']
existing_categorical_columns = [col for col in categorical_columns if col in filtered_data.columns]

if existing_categorical_columns:
    filtered_data = pd.get_dummies(filtered_data, columns=existing_categorical_columns, drop_first=True)

# Select features and target variable
features = ['clicks'] + \
           [col for col in filtered_data.columns if col.startswith('placement_slot_')] + \
           [col for col in filtered_data.columns if col.startswith('targeting_secondary_')] + \
           [col for col in filtered_data.columns if col.startswith('month_name_')] + \
           [col for col in filtered_data.columns if col.startswith('day_name_')]

target = 'impressions'

X = filtered_data[features]
y = filtered_data[target]

# Handle missing values using SimpleImputer
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

if len(numeric_features) > 0:
    numeric_imputer = SimpleImputer(strategy='mean')
    X[numeric_features] = numeric_imputer.fit_transform(X[numeric_features])

if len(categorical_features) > 0:
    categorical_imputer = SimpleImputer(strategy='most_frequent')
    X[categorical_features] = categorical_imputer.fit_transform(X[categorical_features])

# Check if any missing values remain
print("Missing values in X after imputation:")
print(X.isna().sum().sum())

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Gradient Boosting Regressor
gbr = GradientBoostingRegressor(random_state=42)
gbr.fit(X_train, y_train)

# Predict on the test set
y_pred = gbr.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Display the evaluation metrics
evaluation_metrics = {
    'Mean Absolute Error': mae,
    'Mean Squared Error': mse,
    'R-squared': r2
}

print("Evaluation Metrics:")
print(pd.DataFrame([evaluation_metrics]))

# Display feature importances
feature_importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': gbr.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nFeature Importance for Gradient Boosting Regressor:")
pd.set_option('display.max_rows', None)  # Ensure all rows are displayed
print(feature_importance_df)

Columns in Campaign Data:
Index(['ADV_ID', 'AD_ID', 'SKU', 'placement_slot', 'impressions', 'clicks',
       'auction_cost', 'adjusted_cost', 'hit_day_utc', 'targeting_secondary',
       'vertical', 'sub_vertical', 'ntb_click_attributed_units_sold',
       'ntb_view_attributed_units_sold', 'ntb_click_attributed_orders',
       'ntb_view_attributed_orders', 'view_attributed_units_sold',
       'view_attributed_orders', 'hit_day_utc_parsed', 'month_name',
       'day_name'],
      dtype='object')


/var/folders/80/ghdt2w012_x2yg018vx45m300000gn/T/ipykernel_61538/547105180.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numeric_features] = numeric_imputer.fit_transform(X[numeric_features])


Missing values in X after imputation:
0
Evaluation Metrics:
   Mean Absolute Error  Mean Squared Error  R-squared
0          1655.314385        7.271109e+06   0.653224

Feature Importance for Gradient Boosting Regressor:
                                               Feature  Importance
1                     placement_slot_offsite_mobileapp    0.348127
922  targeting_secondary_category=Women's Running S...    0.245414
0                                               clicks    0.116346
346     targeting_secondary_category=Foundation Makeup    0.053019
966    targeting_secondary_content-category=Soap Opera    0.035557
2                     placement_slot_offsite_mobileweb    0.023170
975                                   month_name_March    0.017375
611    targeting_secondary_category=Oral Care Products    0.015794
742  targeting_secondary_category=Single-Serve Coff...    0.013977
722  targeting_secondary_category=Sheet & Pillowcas...    0.013804
424        targeting_secondary_category=Ho

In [5]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
print(feature_importance_df)

                                                                            Feature  \
1                                                  placement_slot_offsite_mobileapp   
922                              targeting_secondary_category=Women's Running Shoes   
0                                                                            clicks   
346                                  targeting_secondary_category=Foundation Makeup   
966                                 targeting_secondary_content-category=Soap Opera   
2                                                  placement_slot_offsite_mobileweb   
975                                                                month_name_March   
611                                 targeting_secondary_category=Oral Care Products   
742                targeting_secondary_category=Single-Serve Coffee Capsules & Pods   
722                            targeting_secondary_category=Sheet & Pillowcase Sets   
424                                     tar